In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Neural Network Dependency Parser Demo\n",
    "\n",
    "This notebook demonstrates how to use the trained neural dependency parser to analyze sentences and visualize dependency trees."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import sys\n",
    "import os\n",
    "sys.path.append(os.path.dirname(os.path.abspath('.')))\n",
    "\n",
    "import torch\n",
    "import pickle\n",
    "import spacy\n",
    "import matplotlib.pyplot as plt\n",
    "import networkx as nx\n",
    "from models.parser import DependencyParser\n",
    "from models.vocab import Vocab\n",
    "from train import load_vocabs\n",
    "\n",
    "print(\"Dependencies loaded successfully!\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. Load the Trained Model"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def load_model_and_vocabs():\n",
    "    \"\"\"Load the trained model and vocabularies.\"\"\"\n",
    "    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')\n",
    "    proc_dir = os.path.join('data', 'processed')\n",
    "    \n",
    "    # Load vocabularies\n",
    "    word_vocab, pos_vocab, label_vocab = load_vocabs(proc_dir)\n",
    "    \n",
    "    # Initialize model\n",
    "    model = DependencyParser(\n",
    "        vocab_sizes={'word': len(word_vocab), 'pos': len(pos_vocab)},\n",
    "        emb_dims={'word': 100, 'pos': 32},\n",
    "        lstm_dim=256,\n",
    "        num_labels=len(label_vocab)\n",
    "    ).to(device)\n",
    "    \n",
    "    # Load trained weights\n",
    "    model.load_state_dict(torch.load('best_model.pt', map_location=device))\n",
    "    model.eval()\n",
    "    \n",
    "    return model, word_vocab, pos_vocab, label_vocab, device\n",
    "\n",
    "# Load model\n",
    "print(\"Loading model and vocabularies...\")\n",
    "model, word_vocab, pos_vocab, label_vocab, device = load_model_and_vocabs()\n",
    "print(f\"Model loaded successfully on {device}!\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. Define Helper Functions"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def tokenize_sentence(sentence):\n",
    "    \"\"\"Tokenize and POS tag a sentence using spaCy.\"\"\"\n",
    "    try:\n",
    "        nlp = spacy.load(\"en_core_web_sm\")\n",
    "    except OSError:\n",
    "        print(\"spaCy English model not found. Installing...\")\n",
    "        os.system(\"python -m spacy download en_core_web_sm\")\n",
    "        nlp = spacy.load(\"en_core_web_sm\")\n",
    "    \n",
    "    doc = nlp(sentence)\n",
    "    words = [token.text for token in doc]\n",
    "    pos_tags = [token.pos_ for token in doc]\n",
    "    return words, pos_tags\n",
    "\n",
    "def predict_dependencies(model, words, pos_tags, word_vocab, pos_vocab, label_vocab, device):\n",
    "    \"\"\"Predict dependency heads and labels for a sentence.\"\"\"\n",
    "    # Convert to indices\n",
    "    word_idx = [word_vocab.get(w, word_vocab['<unk>']) for w in words]\n",
    "    pos_idx = [pos_vocab.get(p, pos_vocab['<unk>']) for p in pos_tags]\n",
    "    \n",
    "    # Convert to tensors\n",
    "    word_tensor = torch.tensor([word_idx], dtype=torch.long).to(device)\n",
    "    pos_tensor = torch.tensor([pos_idx], dtype=torch.long).to(device)\n",
    "    \n",
    "    # Predict\n",
    "    with torch.no_grad():\n",
    "        head_scores, label_scores = model(word_tensor, pos_tensor)\n",
    "        \n",
    "        # Get predictions\n",
    "        pred_heads = head_scores.argmax(-1).squeeze(0)  # (seq_len,)\n",
    "        pred_labels = label_scores.permute(0,2,3,1).gather(\n",
    "            2, pred_heads.unsqueeze(-1).unsqueeze(-1).expand(-1,-1,1,label_scores.size(1))\n",
    "        ).squeeze(2).argmax(-1).squeeze(0)  # (seq_len,)\n",
    "    \n",
    "    return pred_heads.cpu().numpy(), pred_labels.cpu().numpy()\n",
    "\n",
    "def visualize_dependency_tree(words, pos_tags, heads, labels, label_vocab, title=\"Dependency Tree\"):\n",
    "    \"\"\"Visualize the dependency tree using networkx and matplotlib.\"\"\"\n",
    "    # Create graph\n",
    "    G = nx.DiGraph()\n",
    "    \n",
    "    # Add nodes\n",
    "    for i, (word, pos) in enumerate(zip(words, pos_tags)):\n",
    "        G.add_node(i, word=word, pos=pos)\n",
    "    \n",
    "    # Add edges\n",
    "    for i, (head, label) in enumerate(zip(heads, labels)):\n",
    "        if head < len(words):  # Valid head index\n",
    "            label_name = label_vocab.itos[label] if label < len(label_vocab.itos) else \"UNK\"\n",
    "            G.add_edge(head, i, label=label_name)\n",
    "    \n",
    "    # Create layout\n",
    "    pos = nx.spring_layout(G, k=3, iterations=50)\n",
    "    \n",
    "    # Draw the graph\n",
    "    plt.figure(figsize=(12, 8))\n",
    "    nx.draw(G, pos, with_labels=True, node_color='lightblue', \n",
    "            node_size=2000, font_size=10, font_weight='bold',\n",
    "            arrows=True, arrowstyle='->', arrowsize=20)\n",
    "    \n",
    "    # Add edge labels\n",
    "    edge_labels = nx.get_edge_attributes(G, 'label')\n",
    "    nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=8)\n",
    "    \n",
    "    # Add node labels\n",
    "    node_labels = {i: f\"{G.nodes[i]['word']}\\n({G.nodes[i]['pos']})\" for i in G.nodes()}\n",
    "    nx.draw_networkx_labels(G, pos, node_labels, font_size=8)\n",
    "    \n",
    "    plt.title(title)\n",
    "    plt.axis('off')\n",
    "    plt.tight_layout()\n",
    "    plt.show()\n",
    "\n",
    "def compare_with_spacy(sentence):\n",
    "    \"\"\"Compare our parser with spaCy's parser.\"\"\"\n",
    "    print(f\"Sentence: {sentence}\")\n",
    "    print(\"=\"*60)\n",
    "    \n",
    "    # Our parser\n",
    "    print(\"\\nOur Neural Parser:\")\n",
    "    words, pos_tags = tokenize_sentence(sentence)\n",
    "    heads, labels = predict_dependencies(model, words, pos_tags, word_vocab, pos_vocab, label_vocab, device)\n",
    "    \n",
    "    print(\"Dependencies:\")\n",
    "    for i, (word, pos, head, label) in enumerate(zip(words, pos_tags, heads, labels)):\n",
    "        head_word = words[head] if head < len(words) else \"ROOT\"\n",
    "        label_name = label_vocab.itos[label] if label < len(label_vocab.itos) else \"UNK\"\n",
    "        print(f\"  {word} ({pos}) -> {head_word} ({label_name})\")\n",
    "    \n",
    "    # spaCy parser\n",
    "    print(\"\\nspaCy Parser:\")\n",
    "    nlp = spacy.load(\"en_core_web_sm\")\n",
    "    doc = nlp(sentence)\n",
    "    \n",
    "    print(\"Dependencies:\")\n",
    "    for token in doc:\n",
    "        print(f\"  {token.text} ({token.pos_}) -> {token.head.text} ({token.dep_})\")\n",
    "    \n",
    "    # Visualize our parser\n",
    "    visualize_dependency_tree(words, pos_tags, heads, labels, label_vocab, \"Our Neural Parser\")\n",
    "    \n",
    "    # Visualize spaCy parser\n",
    "    spacy_words = [token.text for token in doc]\n",
    "    spacy_pos = [token.pos_ for token in doc]\n",
    "    spacy_heads = [token.head.i for token in doc]\n",
    "    spacy_labels = [token.dep_ for token in doc]\n",
    "    \n",
    "    # Create a simple label mapping for visualization\n",
    "    spacy_label_vocab = type('obj', (object,), {\n",
    "        'itos': list(set(spacy_labels))\n",
    "    })()\n",
    "    spacy_label_indices = [spacy_label_vocab.itos.index(label) for label in spacy_labels]\n",
    "    \n",
    "    visualize_dependency_tree(spacy_words, spacy_pos, spacy_heads, spacy_label_indices, spacy_label_vocab, \"spaCy Parser\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. Test Example Sentences"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Test sentences\n",
    "test_sentences = [\n",
    "    \"The cat sat on the mat.\",\n",
    "    \"I love neural networks.\",\n",
    "    \"She quickly ran to the store.\",\n",
    "    \"The beautiful red car drove fast.\",\n",
    "    \"John gave Mary a book yesterday.\"\n",
    "]\n",
    "\n",
    "for sentence in test_sentences:\n",
    "    compare_with_spacy(sentence)\n",
    "    print(\"\\n\" + \"=\"*80 + \"\\n\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4. Interactive Parsing"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def parse_custom_sentence():\n",
    "    \"\"\"Parse a custom sentence entered by the user.\"\"\"\n",
    "    sentence = input(\"Enter a sentence to parse: \")\n",
    "    if sentence.strip():\n",
    "        compare_with_spacy(sentence)\n",
    "\n",
    "# Uncomment the line below to enable interactive parsing\n",
    "# parse_custom_sentence()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 5. Performance Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def analyze_parser_performance():\n",
    "    \"\"\"Analyze the parser's performance on different sentence types.\"\"\"\n",
    "    sentence_types = {\n",
    "        \"Simple\": [\"The dog barked.\", \"I am happy.\", \"She sings.\"],\n",
    "        \"Complex\": [\"The cat that sat on the mat is sleeping.\", \n",
    "                    \"I believe that he will come tomorrow.\",\n",
    "                    \"The book that I read yesterday was interesting.\"],\n",
    "        \"Questions\": [\"What did you eat?\", \"Where is the cat?\", \"How are you?\"],\n",
    "        \"Negatives\": [\"I don't like it.\", \"She won't come.\", \"They can't see.\"]\n",
    "    }\n",
    "    \n",
    "    for sentence_type, sentences in sentence_types.items():\n",
    "        print(f\"\\n{sentence_type} Sentences:\")\n",
    "        print(\"-\" * 40)\n",
    "        for sentence in sentences:\n",
    "            print(f\"\\nParsing: {sentence}\")\n",
    "            words, pos_tags = tokenize_sentence(sentence)\n",
    "            heads, labels = predict_dependencies(model, words, pos_tags, word_vocab, pos_vocab, label_vocab, device)\n",
    "            \n",
    "            # Print key dependencies\n",
    "            for i, (word, head, label) in enumerate(zip(words, heads, labels)):\n",
    "                if head < len(words):\n",
    "                    head_word = words[head]\n",
    "                    label_name = label_vocab.itos[label] if label < len(label_vocab.itos) else \"UNK\"\n",
    "                    print(f\"  {word} -> {head_word} ({label_name})\")\n",
    "\n",
    "# Uncomment to run performance analysis\n",
    "# analyze_parser_performance()"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.8.0"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}